<a href="https://colab.research.google.com/github/royn5618/AgenticExperiments/blob/main/Agno/Agent_Data_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goal: Test an agent who can analyse and answer questions on an input CSV.

Not Completed - Producing Errors

# Imports

In [ ]:
!pip install -U sqlalchemy 'psycopg[binary]' pgvector
!pip install agno

# API Keys Set-Up

In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

# Set-Up Vector Database

In [ ]:
def udocker_init():
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

# Set-Up PgVector Database

# 1. Install udocker
!udocker --allow-root install

# 2. Kill existing processes and clean up
!pkill -9 -f postgres
!rm -rf /content/pgdata
!udocker --allow-root rm pgvector
!rm -f postgres.log

# 3. Create fresh directory
!mkdir -p /content/pgdata
!chmod -R 777 /content/pgdata

# 4. Pull and create container with correct image path
!udocker --allow-root pull ankane/pgvector
!udocker --allow-root create --name=pgvector ankane/pgvector

# 5. Run the container
!nohup udocker --allow-root run \
    --env="POSTGRES_DB=ai" \
    --env="POSTGRES_USER=ai" \
    --env="POSTGRES_PASSWORD=ai" \
    --env="PGDATA=/var/lib/postgresql/data/pgdata" \
    --volume="/content/pgdata:/var/lib/postgresql/data" \
    --publish="5532:5432" \
    pgvector > postgres.log 2>&1 &

# 6. Connection testing
import time
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

def test_db_connection(max_retries=5, wait_time=10):
    db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

    for attempt in range(max_retries):
        try:
            print(f"\nConnection attempt {attempt + 1}/{max_retries}")
            engine = create_engine(db_url)
            with engine.connect() as connection:
                result = connection.execute(text("SELECT version();"))
                version = result.fetchone()[0]
                print("✅ Successfully connected to PostgreSQL!")
                print(f"Server Version: {version}")

                # Test vector extension
                connection.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
                print("✅ Vector extension ready!")
                return engine
        except OperationalError as e:
            print(f"Attempt {attempt + 1} failed, waiting {wait_time} seconds...")
            print("\nChecking postgres status:")
            !ps aux | grep postgres
            print("\nLatest logs:")
            !tail -n 20 postgres.log
            time.sleep(wait_time)

    return None

# 7. Apply 30 secs sleep time to wait for DB to finish set up before testing for connection
print("Waiting for database to initialize...")
time.sleep(30)

engine = test_db_connection()

if engine:
    print("\n✅ Database is ready for RAG Agent initialization!")
else:
    print("\n❌ Database connection failed. Please check the logs above.")

# Data Analyst Agent

In [ ]:
from agno.agent import Agent
from agno.knowledge.csv import CSVKnowledgeBase
from agno.vectordb.pgvector import PgVector
from agno.models.openai import
from agno.embedder.google import GeminiEmbedder

db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

knowledge_base = CSVKnowledgeBase(
    path="Data/airlines_flights_data.csv",
    vector_db=PgVector(
        table_name="csv_documents",
        db_url=db_url,
        embedder=GeminiEmbedder(),
    ),
    num_documents=5,  # Number of documents to return on search
)


In [ ]:
# Load the knowledge base
knowledge_base.load(recreate=False)

In [ ]:
# Initialize the Agent with the knowledge_base
agent = Agent(
    model=Gemini("gemini-1.5-flash"),
    knowledge=knowledge_base,
    search_knowledge=True,
)